In [ ]:
# load data
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.np_utils import to_categorical
import numpy as np
from keras.models import load_model
from keras.layers import Dense, Convolution2D, BatchNormalization, MaxPooling2D, Input, Flatten
from keras.models import Sequential
import copy
import keras.backend as K
import h5py
from keras.optimizers import SGD

# load model and data
from keras.applications.vgg16 import VGG16
from keras.layers import Dense
from keras.models import Model

def get_model():
    model = VGG16(include_top=True, weights='imagenet')
    out = Dense(2, activation='softmax', name='decision')(model.get_layer('fc2').output)

    model = Model(model.input, out)


    for layer in model.layers:
        print(layer.name)
        layer.trainable=False

    model.layers[-1].trainable=True
    return model

def data_gen(path, batch_size=20):
    batch = []
    labels = []
    while 1:
        f = open(path)
        for line in f:
            img_path, label = line.strip().split(' ')
            img = load_img(img_path, target_size=(224, 224))
            img = img_to_array(img)
            batch.append(img)
            labels.append(int(label))
            if len(batch)==batch_size:
                batch = preprocess_input(np.array(batch))
                labels = to_categorical(np.array(labels))
                yield((batch, labels))
                batch = []
                labels =[]
        f.close()
        
def categorical_to_scalar(cat):
    labels = np.zeros(cat.shape[0])
    for i in range(cat.shape[0]):
        labels[i] = np.argmax(cat[i])
    return labels

def MoveNetwork(net, layer_before_flatten=None):
    """
    This function will extract a subnetwork from the original network, only work with plain networks.
    A plain network is a network with only one input and one output, contains only:
        - Convolution2D
        - MaxPooling2D/AveragePooling2D
        - BatchNormalization
        - Dropout
        - Flatten
        - Dense
        - Activation
    Neurons whose parameters contain at least a non-zero value would be extracted.

    params:
        - net: A Keras model
        - layer_before_flatten: str, the name of the convolution layer just before the flatten layer.
    returns:
        - A sub-network extracted from the original one.
    """
    model = Sequential()
    weight_list = []
    for layer in net.layers:
        if layer.__class__.__name__ == "InputLayer":
            continue
        if layer.__class__.__name__ == 'Convolution2D':
            if len(model.layers)==0:
                w,b = layer.get_weights()
                save_id = []
                for fid in range(w.shape[3]):
                    if np.sum(np.abs(w[:,:,:,fid]))!=0.:
                        save_id.append(fid)
                w_new = np.zeros(w.shape[:3]+(len(save_id),))
                b_new = np.zeros(len(save_id))

                i = 0
                for fid in save_id:
                    w_new[:,:,:,i] = w[:,:,:,fid]
                    b_new[i] = b[fid]
                    i+=1
                config = layer.get_config()
                new_layer = Convolution2D(len(save_id), config['nb_col'], config['nb_row'],
                                          activation=config['activation'], border_mode=config['border_mode'],
                                          name=config['name'], input_shape=layer.input_shape[1:])
                weight_list.append([w_new, b_new])
                model.add(new_layer)
                prev_save_id = copy.deepcopy(save_id)
            else:
                nb_channels = len(prev_save_id)
                w,b = layer.get_weights()
                save_id = []
                for fid in range(w.shape[3]):
                    if np.sum(np.abs(w[:,:,:,fid]))>0.0000001:
                        save_id.append(fid)

                w_new = np.zeros(w.shape[:2]+(nb_channels,len(save_id)))
                b_new = np.zeros(len(save_id))

                i = 0
                for fid in save_id:
                    w_new[:,:,:,i] = w[:,:,prev_save_id,fid]
                    b_new[i] = b[fid]
                    i+=1
                config = layer.get_config()
                new_layer = Convolution2D(len(save_id), config['nb_col'], config['nb_row'],
                          activation=config['activation'], border_mode=config['border_mode'],
                          name=config['name'])
                weight_list.append([w_new, b_new])
                model.add(new_layer)
                prev_save_id = copy.deepcopy(save_id)

        elif layer.__class__.__name__ == 'BatchNormalization':
            gamma, beta, mean, std = layer.get_weights()
            nb_channels = len(prev_save_id)
            gamma_new = np.zeros(len(prev_save_id))
            beta_new = np.zeros(len(prev_save_id))
            mean_new = np.zeros(len(prev_save_id))
            std_new = np.zeros(len(prev_save_id))


            gamma_new = gamma[prev_save_id]
            beta_new = beta[prev_save_id]
            mean_new = mean[prev_save_id]
            std_new = std[prev_save_id]


            config = layer.get_config()
            new_layer = BatchNormalization(name=config['name'], axis=config['axis'])
            weight_list.append([gamma_new, beta_new, mean_new, std_new])
            model.add(new_layer)
            
        elif layer.__class__.__name__ == 'MaxPooling2D' or layer.__class__.__name__ == 'AveragePooling2D':
            config = layer.get_config()
            new_layer = MaxPooling2D(pool_size = config['pool_size'], strides = config['pool_size'], name = config['name'])
            model.add(new_layer)
            weight_list.append(None)

        elif layer.__class__.__name__=='Flatten':
            new_layer = Flatten(name = 'flatten')
            model.add(new_layer)
            weight_list.append(None)

        elif layer.__class__.__name__=='Dense':
            if model.layers[-1].__class__.__name__=='Flatten':
                last_layer = net.get_layer(layer_before_flatten)
                last_layer_output_shape = K.int_shape(last_layer.output)[1:]
                fake_input = np.zeros(last_layer_output_shape)
                fake_input[:,:,prev_save_id] = 1.
                fake_input = np.expand_dims(fake_input, axis=0)

                input_tensor = K.placeholder(fake_input.shape)
                output_tensor = Flatten()(input_tensor)
                get_indicator = K.function([input_tensor, K.learning_phase()],
                          [output_tensor])
                indicators = get_indicator([fake_input, 0])[0]
                indicators = np.squeeze(indicators)
                indicator_index = []
                for i in range(indicators.shape[0]):
                    if indicators[i]==1.:
                        indicator_index.append(i)
                save_id = []
                w,b = layer.get_weights()
                
                for i in range(w.shape[1]):
                    if np.sum(np.abs(w[:,i]))!=0.:
                        save_id.append(i)
                w_new = w[indicator_index,:]
                w_new = w_new[:,save_id]
                b_new = b[save_id]
                config = layer.get_config()
                new_layer = Dense(len(save_id), activation=config['activation'], name=config['name'])
                weight_list.append([w_new,b_new])
                model.add(new_layer)
                prev_save_id = copy.deepcopy(save_id)
                

            else:
                w,b = layer.get_weights()
                save_id = []
                for i in range(w.shape[1]):
                    if np.sum(np.abs(w[:,i]))!=0.:
                        save_id.append(i)
                w_new = w[:,save_id]
                b_new = b[save_id]
                w_new = w_new[prev_save_id,:]

                config = layer.get_config()
                new_layer = Dense(len(save_id), activation=config['activation'], name=config['name'])
                weight_list.append([w_new,b_new])
                model.add(new_layer)
                prev_save_id = copy.deepcopy(save_id)

            

        elif layer.__class__.__name__=='Dropout':
            config = layer.get_config()
            new_layer = Dropout(config['p'])
            weight_list.append(None)
            model.add(new_layer)

            
        else:
            print("Exist unknown layer type:%s"%layer.name)


    for i in range(len(model.layers)):
        if weight_list[i] == None:
            continue
        model.layers[i].set_weights(weight_list[i])
    return model


def next_layer(target_layer):
    """
    This function is set to find the BatchNormalization after the conv layers and fc layers
    you may always need to define this function if your network contains BatchNormalization layer

    """
    next_layers={"block1_conv1":"block1_bn1",
               "block1_conv2":"block1_bn2",
               "block2_conv1":"block2_bn1",
               "block2_conv2":"block2_bn2",
               "block3_conv1":"block3_bn1",
               "block3_conv2":"block3_bn2",
               "block3_conv3":"block3_bn3",
               "block4_conv1":"block4_bn1",
               "block4_conv2":"block4_bn2",
               "block4_conv3":"block4_bn3",
               "block5_conv1":"block5_bn1",
               "block5_conv2":"block5_bn2",
               "block5_conv3":"block5_bn3",
               "fc1":"fc_bn1"
          }
    return next_layers[target_layer]



def get_activation(model, prefix, X_train, y_train):
    """
    This function obtain the activation of each layer when using data-dependent neuron selection strategy and
    save them in several hdf5 files.
    params:
        model: the network
        prefix: the name
    """

    for target_layer in model.layers:
        if target_layer.__class__.__name__ in ["Convolution2D", "Dense"]:
            input_tensor = model.input
            output_tensor = target_layer.output

            get_outputs = K.function([input_tensor, K.learning_phase()],
                              [output_tensor])

            raw_stat = {}
            for s in range(X_train.shape[0]):
                img = np.expand_dims(X_train[s], axis=0)
                label = y_train[s]
                output = get_outputs([img, 0])[0]
                output = np.squeeze(output)
                if target_layer.__class__.__name__=='Dense':
                    output = output
                elif target_layer.__class__.__name__=='Convolution2D':
                    output = np.mean(output, axis = (0,1))
                assert(len(output.shape)==1)

                if raw_stat.has_key(int(label)):
                    raw_stat[int(label)].append(output)
                else:
                    raw_stat[int(label)]= [output]


            f = h5py.File(save_name,'w')
            for k in raw_stat.keys():
                f.create_dataset(name=str(k), data=raw_stat[k])
            f.close()


class NetScore(object):
    """
    This class record the contribution score of all neurons in network. And provide a series of
    methods to get the redundant neurons.

    You may set some properties when instantiating.

    protect_dict: dict, mapping layer_name -> nb_neurons to be protected. If given, at least 'protect_dict[layer_name]'' neurons
                  at 'layer_name' would be preserved.
    model: str, "global" or "layer_wise", default "global". If set to "layer_wise", the number of redundant neurons in each layer
                will be determined by a same pruning ratio or the number given by a dict.
                if set to "global", the number of redundant neurons in each layer will be determined according to the score.

    The methods are:

        add(layer_name, neuron_id, score): add an neuron in the neuron list.
            params:
                layer_name: str, the name of layer that contains the neuron
                neuron_id: int, the index of the neuron in this layer
                score: float, the contribution score of the layer

        sort(): sort the neurons according the contribution score.
        get_redundant_neurons(p): return redundant neurons
            params:
                p: float or dict, if dict, the mode must be layer_wise. if float(0.0~1.0), in layer_wise mode, the redundant neurons
                   in each layer will be selected according to this parameter. In "global" mode, this is the overall pruning ratio
                   of the whole network.
        print_info(): print infos.
    """

    def __init__(self, protect_dict=None, mode="global"):
        self.netscores = {}
        self.sorted = False
        self.total_num = 0
        self.protected_num = 0
        self.neuron_num = {}
        self.prune_num = {}
        self.protect_dict = protect_dict
        self.mode = mode
        self.redundant = None

    def add(self, layer_name, neuron_id, score):
        if self.netscores.has_key(layer_name):
            self.netscores[layer_name].append((layer_name, neuron_id, score))
            self.neuron_num[layer_name] += 1
        else:
            self.netscores[layer_name] = [(layer_name, neuron_id, score)]
            self.neuron_num[layer_name] = 1

        self.total_num += 1

    def sort(self):
        if self.mode == "layer_wise":
            for k in self.netscores.keys():
                self.netscores[k] = sorted(self.netscores[k], key=lambda x:x[2])
            self.sorted = True
        elif self.mode == "global":
            netscores = []
            if self.protect_dict != None:
                for k in self.netscores.keys():
                    self.netscores[k] = sorted(self.netscores[k], key=lambda x:x[2])
                    self.netscores[k] = self.netscores[k][:-self.protect_dict[k]]
            for k in self.netscores.keys():
                netscores += self.netscores[k]
            self.netscores = sorted(netscores, key=lambda x:x[2])
            self.protected_num = self.total_num - len(self.netscores)

    def get_redundant_neurons(self, p):
        if isinstance(p, float):
            if self.mode == "layer_wise":
                if not self.sorted:
                    self.sort()
                redundant = []
                for k in self.netscores.keys():
                    redundant += self.netscores[k][:np.round(self.neuron_num[k]*p)]

                self.redundant = redundant
            elif self.mode == "global":
                self.redundant = self.netscores[:int(np.round(self.total_num*p))]

        if isinstance(p, dict):
            if self.mode == "layer_wise":
                if not self.sorted:
                    self.sort()
                redundant = []

                for k in self.netscores.keys():
                    redundant += self.netscores[k][:p[k]]
                self.redundant = redundant
            elif self.mode == "global":
                raise TypeError("dict only work for global mode!")
                    

        return self.redundant

    def print_info(self):
        for neuron in self.redundant:
            layer_name, neuron_id, score = neuron
            if self.prune_num.has_key(layer_name):
                self.prune_num[layer_name] += 1
            else:
                self.prune_num[layer_name] = 1
        if self.protect_dict == None:
            for k in self.prune_num.keys():
                print("%s: pruned:%d, protected:%d"%(k, self.prune_num[k], 0))
        else:
            for k in self.prune_num.keys():
                print("%s: pruned:%d, protected:%d"%(k, self.prune_num[k], self.protect_dict[k]))
        
        print("pruning: %d -> %d, %d get pruned"%(self.total_num, self.total_num-len(self.redundant), len(self.redundant)))


def get_num_by_ratio(neuron_num,p):
    """
    this function returns the dictionary according to pruning ratio.
    this function should use with mode="layer_wise" and the case when p is a dict.
    We use np.round to erase the errors introduced by integer division
    """
    total_num = 0
    for k in neuron_num:
        total_num += neuron_num[k]
    total_pruned_num = np.round(total_num*p)
    for k in neuron_num:
        neuron_num[k] = int(np.round(total_pruned_num*(neuron_num[k]/float(total_num))))
    return neuron_num







In [2]:
method = "AAWS"
mode = "global"
#pruning_ratio = 0.03
for pruning_ratio in [0.015]:
    prune_round = 19
    target_layers = ["block1_conv1","block1_conv2",
                   "block2_conv1","block2_conv2",
                   "block3_conv1","block3_conv2","block3_conv3",
                   "block4_conv1","block4_conv2","block4_conv3",
                   "block5_conv1","block5_conv2","block5_conv3","fc1", "fc2"] 

    while True:
        model_name = "vgg_catdog_"+str(prune_round-1)+".h5"
        model = load_model(model_name)
        protect_dict = {"block1_conv1":32,
                        "block1_conv2":32,
                       "block2_conv1":64,
                        "block2_conv2":64,
                       "block3_conv1":128,
                        "block3_conv2":128,
                        "block3_conv3":128,
                       "block4_conv1":256,
                        "block4_conv2":256,
                        "block4_conv3":256,
                       "block5_conv1":256,
                        "block5_conv2":256,
                        "block5_conv3":256,
                        "fc1":512, 
                       "fc2":512}
        model.trainable = True
        netscore = NetScore(mode=mode, protect_dict=protect_dict)

        if method == 'AAWS':
            for target_layer in target_layers:
                if target_layer=='fc1':
                    w,b = model.get_layer("fc2").get_weights()
                    weight_sum = np.mean(np.abs(w), axis=1)
                    weight_sum = weight_sum/np.mean(weight_sum)
                elif target_layer=="fc2":
                    w,b = model.get_layer("decision").get_weights()
                    weight_sum = np.mean(np.abs(w), axis=1)
                    weight_sum = weight_sum/np.mean(weight_sum)
                else:
                    w,b = model.get_layer(target_layer).get_weights()
                    weight_sum = np.mean(np.abs(w), axis=(0,1,2))
                    weight_sum = weight_sum/np.mean(weight_sum)

                for neuron_id in range(weight_sum.shape[0]):
                    netscore.add(target_layer, neuron_id, weight_sum[neuron_id])
        elif method == "mean" or method == "sigma":
            mean_dict = {}
            std_dict = {}
            all_data = {}
            for target_layer in target_layers:

                f = h5py.File("prune"+str(prune_round)+'_'+target_layer+'_catdog.h5', 'r')
                raw_stat = {}

                for k in f.keys():
                    raw_stat[k] = f[k][:]


                all_data[target_layer] = raw_stat
                # put activation together
                act_data = []
                for k in raw_stat.keys():
                    act_data += list(raw_stat[k])

                act_data = np.array(act_data)

                mean_dict[target_layer] = np.mean(act_data, axis=0)
                std_dict[target_layer] = np.std(act_data, axis=0)
                mean_dict[target_layer] = mean_dict[target_layer]/np.mean(mean_dict[target_layer])
                std_dict[target_layer] = std_dict[target_layer]/np.mean(std_dict[target_layer])
            if method == 'sigma':
                for target_layer in target_layers:
                    for neuron_id in range(std_dict[target_layer].shape[0]):
                        netscore.add(target_layer, neuron_id, std_dict[target_layer][neuron_id])

            elif method == 'mean':
                for target_layer in target_layers:
                    for neuron_id in range(mean_dict[target_layer].shape[0]):
                        netscore.add(target_layer, neuron_id, mean_dict[target_layer][neuron_id])
                    
        if mode=='layer_wise':
            neuron_num = netscore.neuron_num
            p = get_num_by_ratio(neuron_num, pruning_ratio)

        else:
            p = pruning_ratio
            
        netscore.sort()
        redundant_neurons = netscore.get_redundant_neurons(p)

        print("Pruning....")
        for neuron in redundant_neurons:
            neuron_layer, neuron_id, neuron_score = neuron
            if neuron_layer=='fc1':
                w,b = model.get_layer("fc2").get_weights()
                w[neuron_id,:] = 0.
                model.get_layer("fc2").set_weights([w,b])

                w,b = model.get_layer(neuron_layer).get_weights()
                w[:, neuron_id] = 0.
                b[neuron_id] = 0.
                model.get_layer(neuron_layer).set_weights([w,b])

            elif neuron_layer=='fc2':
                w,b = model.get_layer("decision").get_weights()
                w[neuron_id,:] = 0.
                model.get_layer("decision").set_weights([w,b])

                w,b = model.get_layer(neuron_layer).get_weights()
                w[:, neuron_id] = 0.
                b[neuron_id] = 0.
                model.get_layer(neuron_layer).set_weights([w,b])
            else:
            # do pruning to target_layer
                w,b = model.get_layer(neuron_layer).get_weights()
                w[:,:,:,neuron_id] = 0.
                b[neuron_id] = 0.
                model.get_layer(neuron_layer).set_weights([w,b])


        netscore.print_info()
        print("Moving Network")
        model = MoveNetwork(model, "block5_pool")



    # fine-tune
        test_gen = data_gen('test.txt')
        train_gen = data_gen("train.txt")
        

        sgd = SGD(lr=0.00001, momentum=0.9, decay=5*1e-4,
                     nesterov=True)
        model.compile(loss = 'categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
        model.fit_generator(train_gen, samples_per_epoch=20000, nb_epoch=2, verbose=1, validation_data=test_gen, nb_val_samples=5000)

        model.save("vgg_catdog_"+str(prune_round)+".h5")

        record = "round:%d nb_neurons:%d\n"%(prune_round, netscore.total_num)
        print(record)
        print("------------------------------")
        prune_round+=1
        model.summary()


Pruning....
block1_conv2: pruned:2, protected:32
fc2: pruned:145, protected:512
block1_conv1: pruned:2, protected:32
block2_conv2: pruned:1, protected:64
block2_conv1: pruned:2, protected:64
pruning: 10154 -> 10002, 152 get pruned
Moving Network
Epoch 1/2
20000/20000 [==============================] - 510s - loss: 0.0366 - acc: 0.9969 - val_loss: 0.1392 - val_acc: 0.9860
Epoch 2/2
20000/20000 [==============================] - 507s - loss: 0.0301 - acc: 0.9978 - val_loss: 0.1310 - val_acc: 0.9878
round:19 nb_neurons:10154

------------------------------
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
block1_conv1 (Convolution2D)     (None, 224, 224, 59)  1652        convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (No

ResourceExhaustedError: OOM when allocating tensor with shape[256,256,3,3]
	 [[Node: Conv2D_45 = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](Relu_50, block3_conv3_W_3/read)]]
	 [[Node: Mean_15/_6110 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_1423_Mean_15", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op u'Conv2D_45', defined at:
  File "/home/gpu3/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/gpu3/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/gpu3/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/gpu3/

In [ ]:
# global

target_layers =["block1_conv1","block1_conv2",
               "block2_conv1","block2_conv2",
               "block3_conv1","block3_conv2","block3_conv3",
               "block4_conv195","block4_conv2","block4_conv3",
               "block5_conv1","block5_conv2","block5_conv3",
               "fc1","fc2"]

mean_dict = {}
std_dict = {}
all_data = {}
for target_layer in target_layers:

    f = h5py.File("prune"+str(1)+'_'+target_layer+'_catdog.h5', 'r')
    raw_stat = {}

    for k in f.keys():
        raw_stat[k] = f[k][:]


    all_data[target_layer] = raw_stat
    # put activation together
    act_data = []
    for k in raw_stat.keys():
        act_data += list(raw_stat[k])

    act_data = np.array(act_data)

    mean_dict[target_layer] = np.mean(act_data, axis=0)
    std_dict[target_layer] = np.std(act_data, axis=0)
    mean_dict[target_layer] = mean_dict[target_layer]/np.mean(mean_dict[target_layer])
    std_dict[target_layer] = std_dict[target_layer]/np.mean(std_dict[target_layer])
    
for method in ['min_sigma', 'min_mean']:
    model = load_model("vgg_catdog_0.h5")
    g = open('global_'+method+'catdog', 'w')
    records= []
    netscore = NetScore()
    if method == 'AWS':
        for target_layer in target_layers:
            if target_layer[:2]=='fc':
                w,b = model.get_layer(next_layer(target_layer)).get_weights()
                weight_sum = np.mean(np.abs(w), axis=1)
                weight_sum = weight_sum/np.mean(weight_sum)
            else:
                w,b = model.get_layer(target_layer).get_weights()
                weight_sum = np.mean(np.abs(w), axis=(0,1,2))
                weight_sum = weight_sum/np.mean(weight_sum)
            for neuron_id in range(weight_sum.shape[0]):
                netscore.add(target_layer, neuron_id, weight_sum[neuron_id])
    elif method == 'min_sigma':
        for target_layer in target_layers:
            for neuron_id in range(std_dict[target_layer].shape[0]):
                netscore.add(target_layer, neuron_id, std_dict[target_layer][neuron_id])

    elif method == 'min_mean':
        for target_layer in target_layers:
            for neuron_id in range(mean_dict[target_layer].shape[0]):
                netscore.add(target_layer, neuron_id, mean_dict[target_layer][neuron_id])
    
    netscore.rank()
    
    for pruning_ratio in np.arange(0.01,0.16,0.01):
        redundant_neurons = netscore.get_redundant_neurons(pruning_ratio)
        for neuron in redundant_neurons:
            neuron_layer, neuron_id, neuron_score = neuron
            if neuron_layer[:2]=='fc':
                w,b = model.get_layer(next_layer(neuron_layer)).get_weights()  
                if method=="min_sigma":
                    b += mean_dict[neuron_layer][neuron_id]*w[neuron_id,:]
                w[neuron_id,:] = 0.
                model.get_layer(next_layer(neuron_layer)).set_weights([w,b]) 

                w,b = model.get_layer(neuron_layer).get_weights()  
                w[:, neuron_id] = 0.
                b[neuron_id] = 0.
                model.get_layer(neuron_layer).set_weights([w,b])

            else:
            # do pruning to target_layer
                w,b = model.get_layer(neuron_layer).get_weights()      
                w[:,:,:,neuron_id] = 0.
                b[neuron_id] = 0.
                model.get_layer(neuron_layer).set_weights([w,b])   

        test_gen = data_gen('test.txt')
        model.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
        score = model.evaluate_generator(test_gen, val_samples=5000)
        records.append(str(pruning_ratio)+' '+str(score[1])+'\n')
        netscore.print_info(pruning_ratio)
        print("------------------------------")

    g.writelines(records)
    g.close()
    print("finished")

In [1]:
from keras.models import load_model
import keras.backend as K
import h5py


prune_round = 4
model_name = "vgg_catdog_"+str(prune_round-1)+".h5"
model = load_model(model_name)


target_layers =["block1_conv1","block1_conv2",
               "block2_conv1","block2_conv2",
               "block3_conv1","block3_conv2","block3_conv3",
               "block4_conv1","block4_conv2","block4_conv3",
               "block5_conv1","block5_conv2","block5_conv3",
               "fc1","fc2"]

for target_layer in target_layers:
    print("processing "+target_layer)

    input_tensor = model.input
    output_tensor = model.get_layer(target_layer).output

    get_outputs = K.function([input_tensor, K.learning_phase()],
                          [output_tensor])

    raw_stat = {}
    count = 0
    train_gen = data_gen('train.txt')
    for batch in train_gen:
        count += 1
        if count%50 == 0:
            print("processing the %dth batch training sample"%count)
        imgs, label = batch
        label = categorical_to_scalar(label)
        output = get_outputs([imgs, 0])[0]
        for i in range(output.shape[0]):
            if len(output[i].shape) == 3:
                output_sum = np.sum(output[i], axis = (0,1))
            else:
                assert(len(output[i].shape)==1)
                output_sum = output[i]
            
            if raw_stat.has_key(label[i]):
                raw_stat[label[i]].append(output_sum)
            else:
                raw_stat[label[i]] = [output_sum]
        if count==500:
            break

        
    f = h5py.File("prune"+str(prune_round)+'_'+target_layer+'_catdog.h5','w')
    for k in raw_stat.keys():
        f.create_dataset(name=str(k), data=raw_stat[k])
    f.close()

Using TensorFlow backend.


processing block1_conv1


NameError: name 'data_gen' is not defined

In [2]:
## no fine-tune

mode = "global"
#pruning_ratio = 0.03

for method in ["AAWS", "mean"]:
    for pruning_ratio in [0., 0.02, 0.04, 0.06, 0.1, 0.12,0.14,0.16,0.18,0.2,0.22]:
        prune_round = 1
        target_layers = ["block1_conv1","block1_conv2",
                   "block2_conv1","block2_conv2",
                   "block3_conv1","block3_conv2","block3_conv3",
                   "block4_conv1","block4_conv2","block4_conv3",
                   "block5_conv1","block5_conv2","block5_conv3","fc1", "fc2"] 

        protect_dict = {"block1_conv1":32,
                        "block1_conv2":32,
                       "block2_conv1":64,
                        "block2_conv2":64,
                       "block3_conv1":128,
                        "block3_conv2":128,
                        "block3_conv3":128,
                       "block4_conv1":256,
                        "block4_conv2":256,
                        "block4_conv3":256,
                       "block5_conv1":256,
                        "block5_conv2":256,
                        "block5_conv3":256,
                        "fc1":512, 
                        "fc2":512}
        model_name = "vgg_catdog_"+str(prune_round-1)+".h5"
        model = load_model(model_name)
        
        model.trainable = True
        netscore = NetScore(mode=mode, protect_dict=protect_dict)

        if method == 'AAWS':
            for target_layer in target_layers:
                if target_layer=='fc1':
                    w,b = model.get_layer("fc2").get_weights()
                    weight_sum = np.mean(np.abs(w), axis=1)
                    weight_sum = weight_sum/np.mean(weight_sum)
                elif target_layer=="fc2":
                    w,b = model.get_layer("decision").get_weights()
                    weight_sum = np.mean(np.abs(w), axis=1)
                    weight_sum = weight_sum/np.mean(weight_sum)
                else:
                    w,b = model.get_layer(target_layer).get_weights()
                    weight_sum = np.mean(np.abs(w), axis=(0,1,2))
                    weight_sum = weight_sum/np.mean(weight_sum)

                for neuron_id in range(weight_sum.shape[0]):
                    netscore.add(target_layer, neuron_id, weight_sum[neuron_id])
        elif method == "mean" or method == "sigma":
            mean_dict = {}
            std_dict = {}
            all_data = {}
            for target_layer in target_layers:

                f = h5py.File("prune"+str(prune_round)+'_'+target_layer+'_catdog.h5', 'r')
                raw_stat = {}

                for k in f.keys():
                    raw_stat[k] = f[k][:]


                all_data[target_layer] = raw_stat
                # put activation together
                act_data = []
                for k in raw_stat.keys():
                    act_data += list(raw_stat[k])

                act_data = np.array(act_data)

                mean_dict[target_layer] = np.mean(act_data, axis=0)
                std_dict[target_layer] = np.std(act_data, axis=0)
                mean_dict[target_layer] = mean_dict[target_layer]/np.mean(mean_dict[target_layer])
                std_dict[target_layer] = std_dict[target_layer]/np.mean(std_dict[target_layer])
            if method == 'sigma':
                for target_layer in target_layers:
                    for neuron_id in range(std_dict[target_layer].shape[0]):
                        netscore.add(target_layer, neuron_id, std_dict[target_layer][neuron_id])

            elif method == 'mean':
                for target_layer in target_layers:
                    for neuron_id in range(mean_dict[target_layer].shape[0]):
                        netscore.add(target_layer, neuron_id, mean_dict[target_layer][neuron_id])
                    
        if mode=='layer_wise':
            neuron_num = netscore.neuron_num
            p = get_num_by_ratio(neuron_num, pruning_ratio)

        else:
            p = pruning_ratio
            
        netscore.sort()
        redundant_neurons = netscore.get_redundant_neurons(p)

        print("Pruning....")
        for neuron in redundant_neurons:
            neuron_layer, neuron_id, neuron_score = neuron
            if neuron_layer=='fc1':
                w,b = model.get_layer("fc2").get_weights()
                w[neuron_id,:] = 0.
                model.get_layer("fc2").set_weights([w,b])

                w,b = model.get_layer(neuron_layer).get_weights()
                w[:, neuron_id] = 0.
                b[neuron_id] = 0.
                model.get_layer(neuron_layer).set_weights([w,b])

            elif neuron_layer=='fc2':
                w,b = model.get_layer("decision").get_weights()
                w[neuron_id,:] = 0.
                model.get_layer("decision").set_weights([w,b])

                w,b = model.get_layer(neuron_layer).get_weights()
                w[:, neuron_id] = 0.
                b[neuron_id] = 0.
                model.get_layer(neuron_layer).set_weights([w,b])
            else:
            # do pruning to target_layer
                w,b = model.get_layer(neuron_layer).get_weights()
                w[:,:,:,neuron_id] = 0.
                b[neuron_id] = 0.
                model.get_layer(neuron_layer).set_weights([w,b])


        netscore.print_info()




    # fine-tune
        test_gen = data_gen('test.txt')
        

        sgd = SGD(lr=0.00001, momentum=0.9, decay=5*1e-4,
                     nesterov=True)
        model.compile(loss = 'categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
        score = model.evaluate_generator(test_gen, 5000)


        record = "method:%s ratio:%f nb_neurons:%d acc:%f\n"%(method, pruning_ratio, netscore.total_num, score[1])
        print(record)
        print("------------------------------")



Pruning....
pruning: 12416 -> 12416, 0 get pruned
method:AAWS ratio:0.000000 nb_neurons:12416 acc:0.982400

------------------------------
Pruning....
fc2: pruned:247, protected:512
block2_conv1: pruned:1, protected:64
pruning: 12416 -> 12168, 248 get pruned
method:AAWS ratio:0.020000 nb_neurons:12416 acc:0.982400

------------------------------
Pruning....
fc2: pruned:495, protected:512
block2_conv1: pruned:2, protected:64
pruning: 12416 -> 11919, 497 get pruned
method:AAWS ratio:0.040000 nb_neurons:12416 acc:0.982400

------------------------------
Pruning....
fc2: pruned:742, protected:512
block2_conv1: pruned:3, protected:64
pruning: 12416 -> 11671, 745 get pruned
method:AAWS ratio:0.060000 nb_neurons:12416 acc:0.982200

------------------------------
Pruning....
fc2: pruned:1236, protected:512
block1_conv1: pruned:2, protected:32
block2_conv1: pruned:4, protected:64
pruning: 12416 -> 11174, 1242 get pruned
method:AAWS ratio:0.100000 nb_neurons:12416 acc:0.982400

-----------------

ResourceExhaustedError: OOM when allocating tensor with shape[20,64,224,224]
	 [[Node: Conv2D_144 = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](Relu_165, block1_conv2_W_11/read)]]
Caused by op u'Conv2D_144', defined at:
  File "/home/gpu3/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/gpu3/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/gpu3/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/gpu3/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 592, in launch_instance
    app.start()
  File "/home/gpu3/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 403, in start
    ioloop.IOLoop.instance().start()
  File "/home/gpu3/anaconda2/lib/pyth